<a href="https://colab.research.google.com/github/ffreirea1988/usm/blob/pega/Carga_gastos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Nombre del archivo (se modifica mes a mes)
nmb_input = 'Cuentas TI - 092023 - Septiembre 2023.xlsx'

In [ ]:
df_2701 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112701')
df_2702 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112702')
df_2703 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112703')
df_2704 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112704')
df_2705 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112705')
df_2710 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112710')
df_2711 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112711')
df_2712 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112712')

In [ ]:
#Concatenar archivos en un solo dataframe con todas las cuentas
#axis=1 unir por filas
#axis=0 unir por columnas
df = pd.concat([df_2701, df_2702, df_2703, df_2704, df_2705, df_2710, df_2711, df_2712], axis=0)